In [1]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.feature import PCA as spark_PCA, StandardScaler as spark_scaler
from pyspark.ml.linalg import Vectors, VectorUDT, SparseVector, DenseVector

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1692171824783_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [3]:
PATH = 's3://kiliandatadev-p8-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
PATH_Result_Demo = PATH+'/Results/Demo'
PATH_Demo = PATH+'/Demo'
print('PATH:             '+\
      PATH+'\nPATH_Data:        '+\
      PATH_Data+'\nPATH_Result:      '+PATH_Result+\
      '\nPATH_Demo:        '+PATH_Demo+\
      '\nPATH_Result_Demo: '+PATH_Result_Demo
     )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:             s3://kiliandatadev-p8-data
PATH_Data:        s3://kiliandatadev-p8-data/Test
PATH_Result:      s3://kiliandatadev-p8-data/Results
PATH_Demo:        s3://kiliandatadev-p8-data/Demo
PATH_Result_Demo: s3://kiliandatadev-p8-data/Results/Demo

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://kiliandatade...|2023-08-14 13:23:56|  7353|[FF D8 FF E0 00 1...|
|s3://kiliandatade...|2023-08-14 13:23:56|  7350|[FF D8 FF E0 00 1...|
|s3://kiliandatade...|2023-08-14 13:23:56|  7349|[FF D8 FF E0 00 1...|
|s3://kiliandatade...|2023-08-14 13:23:56|  7348|[FF D8 FF E0 00 1...|
|s3://kiliandatade...|2023-08-14 13:23:57|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [8]:
images.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22688

In [9]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------+----------+
|path                                                    |label     |
+--------------------------------------------------------+----------+
|s3://kiliandatadev-p8-data/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://kiliandatadev-p8-data/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://kiliandatadev-p8-data/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://kiliandatadev-p8-data/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://kiliandatadev-p8-data/Test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------------+----------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1692171824783_0002/container_1692171824783_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Création de notre dataframe Spark final - conversion des arrays en vecteurs
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_spark_vector = features_df.select(
    features_df["path"],
    features_df["label"], 
    list_to_vector_udf(features_df["features"]).alias("features")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_spark_vector.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|                path|         label|            features|
+--------------------+--------------+--------------------+
|s3://kiliandatade...|Pineapple Mini|[0.0,4.9297246932...|
|s3://kiliandatade...| Cucumber Ripe|[1.88859605789184...|
|s3://kiliandatade...|     Raspberry|[0.01002074219286...|
|s3://kiliandatade...|   Cauliflower|[0.0,0.1326987892...|
|s3://kiliandatade...|   Cauliflower|[0.13888484239578...|
+--------------------+--------------+--------------------+
only showing top 5 rows

In [19]:
# StandardScaler de la librairie Spark
scaler = spark_scaler(
    inputCol = 'features', 
    outputCol = 'scaled_features',
    withMean = True,
    withStd = True
).fit(df_spark_vector)

df_spark_vector = scaler.transform(df_spark_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# ACP de la librairie Spark
spark_pca = spark_PCA(k=26, inputCol="scaled_features")
spark_pca.setOutputCol("pca_features")
spark_pca_model = spark_pca.fit(df_spark_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_spark_vector = spark_pca_model.transform(dataset=df_spark_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
vector_to_array_udf = udf(
    lambda vector: vector.toArray().tolist()
    if isinstance(vector, (DenseVector, SparseVector))
    else vector,
    ArrayType(FloatType())
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# On doit repasser les vecteurs en array pour qu'ils soient lisibles en Python
df_spark_vector = df_spark_vector.withColumn("features", vector_to_array_udf(df_spark_vector["features"]))
df_spark_vector = df_spark_vector.withColumn("pca_features", vector_to_array_udf(df_spark_vector["pca_features"]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_spark_results = df_spark_vector.select(
    df_spark_vector["path"],
    df_spark_vector["label"], 
    # df_spark_vector["features"],
    df_spark_vector["pca_features"]
)

df_spark_results.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: float (containsNull = true)

In [27]:
df_spark_results.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [29]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                       pca_features
0  s3://kiliandatadev-p8-data/Test/Raspberry/244_...  ...  [-2.6057673, -0.3092375, 1.1311083, -18.352468...
1  s3://kiliandatadev-p8-data/Test/Cauliflower/r_...  ...  [-15.943637, 2.387592, 2.8377476, -8.413417, -...
2  s3://kiliandatadev-p8-data/Test/Cauliflower/r2...  ...  [-7.702747, 5.54361, 1.5083392, -11.03542, -12...
3  s3://kiliandatadev-p8-data/Test/Cauliflower/20...  ...  [-8.625718, 4.986315, 3.660708, -10.492062, -1...
4  s3://kiliandatadev-p8-data/Test/Cauliflower/11...  ...  [-8.074036, 4.504894, 2.7043364, -9.0124035, -...

[5 rows x 3 columns]

In [31]:
# Devrait être 26
df.loc[0,'pca_features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(26,)

In [32]:
# Devrait être (22688,3)
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22688, 3)

In [33]:
df.to_csv(path_or_buf=f"{PATH_Result}/results.csv", index=False, sep=",")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…